# Building a Document Indexing Pipeline with Haystack

Welcome to this tutorial notebook on building a document indexing pipeline using Haystack! In this notebook, you'll learn how to create a robust pipeline for processing and indexing documents that can be later used for question answering and information retrieval.

## Learning Objectives
By the end of this notebook, you will:
- Understand how to set up a Haystack document processing pipeline
- Learn to work with different document formats (TXT, CSV, PDF)
- Create and configure an in-memory document store
- Implement document preprocessing and indexing
- Test your indexed documents with basic queries

## Prerequisites
- Basic Python knowledge
- Understanding of basic NLP concepts
- Familiarity with Jupyter notebooks

Let's get started!

In [1]:
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(".env")

True

## 1. Setup and Package Installation

In this section, we'll import the necessary packages for our indexing pipeline. Here's what each package does:

In [2]:
# Import core Haystack classes
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.components.joiners import DocumentJoiner

# Import components for data fetching and conversion
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import (
    PyPDFToDocument,
    TextFileToDocument,
    HTMLToDocument,
)
from haystack.components.routers import FileTypeRouter

# Import components for preprocessing
from haystack.components.preprocessors import (
    DocumentCleaner,
    DocumentSplitter,
    CSVDocumentCleaner,
    CSVDocumentSplitter
)

# Import components for embedding
from haystack.components.embedders import SentenceTransformersDocumentEmbedder


/Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch4/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Document Loading and Preprocessing

In this section, we'll set up our document loading pipeline. We'll work with multiple document formats:

1. Text files (.txt)
2. CSV files
3. PDF documents

The data we'll be working with includes:
- A text file about Haystack's introduction
- A CSV file containing information about LLM models
- A sample PDF document

We'll create a preprocessing pipeline that will:
1. Load these documents from different sources
2. Convert them into a unified format
3. Clean and prepare them for indexing


In [3]:
# --- 1. Create Sample Data Files ---
# Create a directory to hold our source files
data_dir = Path("data_for_indexing")
data_dir.mkdir(exist_ok=True)

# Create a sample text file
text_file_path = data_dir / "haystack_intro.txt"
text_file_path.write_text(
    "Haystack is an open-source framework by deepset for building production-ready LLM applications. "
    "It enables developers to create retrieval-augmented generative pipelines and state-of-the-art search systems."
)

205

Create mock CSV

In [22]:
# Create a sample CSV file with some empty rows/columns for cleaning
csv_content = """Company,Model,Release Year,,Notes
OpenAI,GPT-4,2023,,Generative Pre-trained Transformer 4
,,,
Google,Gemini,2023,,A family of multimodal models
Anthropic,Claude 3,2024,,Includes Opus, Sonnet, and Haiku models
"""
csv_file_path = data_dir / "llm_models.csv"
csv_file_path.write_text(csv_content)


209

Scrape a webpage

In [ ]:
# Define a sample URL to fetch
web_url = "https://haystack.deepset.ai/blog/haystack-2-release"

PDF for file that does not exist.

In [5]:
# For this example, we'll skip the actual PDF creation and assume one exists.
# You can place any PDF file in the 'data_for_indexing' directory and name it 'sample.pdf'.
# For a runnable example, we will simulate its path.
pdf_file_path = data_dir / "sample.pdf"
# In a real scenario, you would have this file. For this script to run, we'll check for it.
if not pdf_file_path.exists():
    print(f"Warning: PDF file not found at {pdf_file_path}. The PDF processing branch will not run.")
    # Create a dummy file to avoid path errors, but it won't be processed as PDF
    pdf_file_path.touch()

## 3. Initialize components

In [20]:
# DocumentStore:
document_store = InMemoryDocumentStore()

# FileTypeRouter: Directs files to the correct converter based on their MIME type.
file_type_router = FileTypeRouter(mime_types=["text/plain", "application/pdf", "text/html", "text/csv"])

# Converters: One for each file type we want to handle.
text_converter = TextFileToDocument()
pdf_converter = PyPDFToDocument()
html_converter = HTMLToDocument()
# Added a dedicated converter instance for CSV files.
# It's also a TextFileToDocument, as it just needs to read the file's content.
csv_converter = TextFileToDocument()


# LinkContentFetcher: Fetches content from URLs.
link_fetcher = LinkContentFetcher()

# DocumentJoiners:
unstructured_doc_joiner = DocumentJoiner()
# This joiner will gather documents from *all* processing branches
# (the split text docs and the split csv docs) before embedding.
final_doc_joiner = DocumentJoiner()

# Preprocessors for Text Data:
text_cleaner = DocumentCleaner()
text_splitter = DocumentSplitter(split_by="word", split_length=150, split_overlap=20)

# Preprocessors for Tabular Data (CSV):
# These will now be part of the main pipeline.
csv_cleaner = CSVDocumentCleaner()
csv_splitter = CSVDocumentSplitter(split_mode="row-wise")

# Embedder:
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

# DocumentWriter:
writer = DocumentWriter(document_store)


## 4. Building the Indexing Pipeline

Now we'll construct our document processing pipeline. 

In [ ]:

indexing_pipeline = Pipeline()

# Add all components to the pipeline with unique names
indexing_pipeline.add_component("link_fetcher", link_fetcher)
indexing_pipeline.add_component("html_converter", html_converter)
indexing_pipeline.add_component("file_type_router", file_type_router)
indexing_pipeline.add_component("text_converter", text_converter)
indexing_pipeline.add_component("pdf_converter", pdf_converter)
indexing_pipeline.add_component("unstructured_doc_joiner", unstructured_doc_joiner)
indexing_pipeline.add_component("text_cleaner", text_cleaner)
indexing_pipeline.add_component("text_splitter", text_splitter)
indexing_pipeline.add_component("doc_embedder", doc_embedder)
indexing_pipeline.add_component("writer", writer)

# **NEW**: Add the CSV components to the pipeline
indexing_pipeline.add_component("csv_converter", csv_converter)
indexing_pipeline.add_component("csv_cleaner", csv_cleaner)
indexing_pipeline.add_component("csv_splitter", csv_splitter)
indexing_pipeline.add_component("final_doc_joiner", final_doc_joiner)

## 5. Connect components

This is where the routing logic will come in. We will first handle web-based data, then local files (TXT and PDF)
in an unstructured data branch. 

In [ ]:

# --- Unstructured Data Branch (Web, TXT, PDF) ---
# Web data
indexing_pipeline.connect("link_fetcher.streams", "html_converter.sources")
indexing_pipeline.connect("html_converter.documents", "unstructured_doc_joiner.documents")

# Local file data (TXT, PDF)
indexing_pipeline.connect("file_type_router.text/plain", "text_converter.sources")
indexing_pipeline.connect("file_type_router.application/pdf", "pdf_converter.sources")
indexing_pipeline.connect("text_converter.documents", "unstructured_doc_joiner.documents")
indexing_pipeline.connect("pdf_converter.documents", "unstructured_doc_joiner.documents")

# Processing for unstructured data
indexing_pipeline.connect("unstructured_doc_joiner", "text_cleaner")
indexing_pipeline.connect("text_cleaner", "text_splitter")
# Connect the split *text* docs to the *final* joiner
indexing_pipeline.connect("text_splitter.documents", "final_doc_joiner.documents")


#  Structured Data Branch (CSV) ---
# Route CSV files to the csv_converter
indexing_pipeline.connect("file_type_router.text/csv", "csv_converter.sources")
# Process the CSV documents
indexing_pipeline.connect("csv_converter.documents", "csv_cleaner.documents")
indexing_pipeline.connect("csv_cleaner.documents", "csv_splitter.documents")
# Connect the split *csv* docs to the *final* joiner
indexing_pipeline.connect("csv_splitter.documents", "final_doc_joiner.documents")


# --- Main Processing Path (Embedding and Writing) ---
# The final_doc_joiner now receives documents from *both* branches
indexing_pipeline.connect("final_doc_joiner", "doc_embedder")
indexing_pipeline.connect("doc_embedder", "writer")

🚅 Components
  - link_fetcher: LinkContentFetcher
  - html_converter: HTMLToDocument
  - file_type_router: FileTypeRouter
  - text_converter: TextFileToDocument
  - pdf_converter: PyPDFToDocument
  - unstructured_doc_joiner: DocumentJoiner
  - text_cleaner: DocumentCleaner
  - text_splitter: DocumentSplitter
  - doc_embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
  - csv_converter: TextFileToDocument
  - csv_cleaner: CSVDocumentCleaner
  - csv_splitter: CSVDocumentSplitter
  - final_doc_joiner: DocumentJoiner
🛤️ Connections
  - link_fetcher.streams -> html_converter.sources (list[ByteStream])
  - html_converter.documents -> unstructured_doc_joiner.documents (list[Document])
  - file_type_router.text/plain -> text_converter.sources (list[Union[str, Path, ByteStream]])
  - file_type_router.application/pdf -> pdf_converter.sources (list[Union[str, Path, ByteStream]])
  - file_type_router.text/csv -> csv_converter.sources (list[Union[str, Path, ByteStream]])
  - t

Let's visualize the pipeline:

In [16]:
indexing_pipeline.draw(path="./images/indexing_pipeline.png")

![](./images/indexing_pipeline.png)

In [17]:
# --- 5. Run the Pipeline ---

print("Running unified indexing pipeline for web, local files, and CSV...")
# Note: The PDF path will be ignored if the file doesn't exist.
file_paths_to_process = [text_file_path]

if pdf_file_path.exists() and pdf_file_path.stat().st_size > 0:
    file_paths_to_process.append(pdf_file_path)
else:
    print(f"Skipping PDF file: {pdf_file_path}")

file_paths_to_process.append(csv_file_path)


indexing_pipeline.run({
    "link_fetcher": {"urls": [web_url]},
    "file_type_router": {"sources": file_paths_to_process}
})

Error processing document 1384ec36dd6d99f90ab589732d5219b7371dac846d0f0bd89c6385189c4079c0. Keeping it, but skipping cleaning. Error: Error tokenizing data. C error: Expected 5 fields in line 5, saw 7

Error processing document 1384ec36dd6d99f90ab589732d5219b7371dac846d0f0bd89c6385189c4079c0. Keeping it, but skipping splitting. Error: Error tokenizing data. C error: Expected 5 fields in line 5, saw 7



Running unified indexing pipeline for web, local files, and CSV...
Skipping PDF file: data_for_indexing/sample.pdf


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


{'writer': {'documents_written': 15}}

## Understanding the Results

Let's examine what our pipeline has processed. We'll look at:
1. The number of documents processed
2. How the documents were split
3. The metadata that was extracted

This will help us verify that our pipeline is working as expected and give us insights into our indexed documents.

First, let's query our document store to see what we've indexed:

In [18]:
# --- 7. Verify the DocumentStore ---
doc_count = document_store.count_documents()
print(f"\nTotal documents in DocumentStore: {doc_count}")
print("Sample document from the store:")
print(document_store.filter_documents())


Total documents in DocumentStore: 15
Sample document from the store:
[Document(id=1384ec36dd6d99f90ab589732d5219b7371dac846d0f0bd89c6385189c4079c0, content: 'Company,Model,Release Year,,Notes
OpenAI,GPT-4,2023,,Generative Pre-trained Transformer 4
,,,
Google...', meta: {'file_path': 'llm_models.csv'}, embedding: vector of size 384), Document(id=be2fb4afe8f3e531ae2e97314778b92789c44d794c981d67bdea8658cf3fe51e, content: 'Haystack 2.0: The Composable Open-Source LLM Framework
Meet Haystack 2.0, a more flexible, customiza...', meta: {'content_type': 'text/html', 'url': 'https://haystack.deepset.ai/blog/haystack-2-release', 'source_id': '0b188f4690ab3496d2270baf378be9fde19707e9b1ece003123c0442af918bb7', 'page_number': 1, 'split_id': 0, 'split_idx_start': 0, '_split_overlap': [{'doc_id': 'ff9dcb8208955755eb458bdf693e12ba8c2dd7f27de0bcb51e47e474255ad47b', 'range': (0, 124)}]}, embedding: vector of size 384), Document(id=ff9dcb8208955755eb458bdf693e12ba8c2dd7f27de0bcb51e47e474255ad47b, conten